In [1]:
# Imports and Setup
import os
import sys
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers, models

sys.path.insert(0, str(Path("..") / "src"))

from utils import config
from utils.io import load_cleaned, load_method_ready, results_dir, append_csv_row, save_json
from utils.evaluation import compute_binary_metrics
from utils.plotting import plot_signal, plot_scores

/Users/apostolos_sterpis/BSP_Anomaly_Detection/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Helper: Windows, Mapping and Model

def windows_from_starts(series: np.ndarray, starts: np.ndarray, win_size: int) -> np.ndarray:
    """Build windows (n, win_size, 1) using the exact start indices."""
    series = np.asarray(series).reshape(-1)
    w = np.lib.stride_tricks.sliding_window_view(series, win_size)  # (n_possible, win_size)
    X = w[starts]  # (n_windows, win_size)
    return X[..., None].astype(np.float32)  # (n_windows, win_size, 1)


def windows_scores_to_point_scores_max(
    win_starts_local: np.ndarray,
    win_size: int,
    scores_win: np.ndarray,
    n_points: int,
) -> np.ndarray:
    """Point score = max window score covering each point."""
    start_scores = np.full(n_points, np.nan, dtype=float)
    for s, sc in zip(win_starts_local, scores_win):
        s = int(s)
        if 0 <= s < n_points:
            start_scores[s] = float(sc)

    from collections import deque
    dq = deque()  # indices with decreasing scores

    out = np.full(n_points, np.nan, dtype=float)
    for i in range(n_points):
        if i < n_points and not np.isnan(start_scores[i]):
            while dq and start_scores[dq[-1]] <= start_scores[i]:
                dq.pop()
            dq.append(i)

        left = i - win_size + 1
        while dq and dq[0] < max(0, left):
            dq.popleft()

        out[i] = start_scores[dq[0]] if dq else np.nan

    return out


def build_lstm_ae(window_size: int, lstm_units: int) -> tf.keras.Model:
    """Simple LSTM autoencoder."""
    inp = layers.Input(shape=(window_size, 1))
    x = layers.LSTM(lstm_units, return_sequences=False)(inp)
    x = layers.RepeatVector(window_size)(x)
    x = layers.LSTM(lstm_units, return_sequences=True)(x)
    out = layers.TimeDistributed(layers.Dense(1))(x)
    return models.Model(inp, out)


def reconstruction_mae(X: np.ndarray, X_hat: np.ndarray) -> np.ndarray:
    """MAE per window."""
    return np.mean(np.abs(X - X_hat), axis=(1, 2))

In [3]:
# Main Execution Loop

tf.random.set_seed(config.RANDOM_SEED)
np.random.seed(config.RANDOM_SEED)

base_out = results_dir("deep_learning")
csv_path = base_out / "deep_learning_results.csv"
if csv_path.exists():
    os.remove(csv_path)

win_size = config.WINDOW_SIZE
margin = config.PLOT_ZOOM_MARGIN

ae_epochs = config.AE_PARAMS["epochs"]
ae_batch = config.AE_PARAMS["batch_size"]
ae_val = config.AE_PARAMS["validation_split"]
ae_patience = config.AE_PARAMS["early_stopping_patience"]
ae_units = config.AE_PARAMS["lstm_units"]
ae_clipnorm = config.AE_PARAMS["clipnorm"]
thr_q = config.AE_PARAMS["threshold_quantile"]

for dataset_name in config.DATASETS:
    # Load point labels + split
    _, labels, meta = load_cleaned(dataset_name)
    train_end = int(meta["train_end"])
    y_test_point = labels[train_end:]

    # Load method-ready
    mr = load_method_ready(dataset_name)
    train_z = mr["train_z"]
    test_z = mr["test_z"]

    train_starts = mr["train_win_starts"]
    test_starts_abs = mr["test_win_starts"]
    y_test_win = mr["test_win_labels"]

    # Convert test starts to local test coordinates
    test_starts = test_starts_abs - train_end

    # Build windows using the exact starts (no stride mismatch)
    X_train = windows_from_starts(train_z, train_starts, win_size)
    X_test = windows_from_starts(test_z, test_starts, win_size)

    if len(X_test) != len(y_test_win):
        raise ValueError(
            f"[{dataset_name}] Window alignment mismatch: "
            f"X_test={len(X_test)} vs test_win_labels={len(y_test_win)}"
        )

    out_dir = results_dir("deep_learning", dataset_name)

    # Overview plot (signal)
    test_raw = mr["test_raw"]
    plot_signal(
        test_raw,
        true_labels=y_test_point,
        title=f"{dataset_name} - Test (overview)",
        save_path=out_dir / "overview_signal.png",
        max_points=5000,
    )

    # Zoom window (test coords)
    a0 = int(meta["anomaly_start"]) - train_end
    a1 = int(meta["anomaly_end"]) - train_end
    z0 = max(0, a0 - margin)
    z1 = min(len(test_raw), a1 + margin)

    # Model
    model = build_lstm_ae(win_size, ae_units)
    opt = tf.keras.optimizers.Adam(clipnorm=ae_clipnorm)
    model.compile(optimizer=opt, loss="mae")

    cb = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=ae_patience, restore_best_weights=True
        )
    ]

    # Train 
    model.fit(
        X_train, X_train,
        epochs=ae_epochs,
        batch_size=ae_batch,
        validation_split=ae_val,
        shuffle=False,
        callbacks=cb,
        verbose=1,
    )

    # Scores (window)
    X_train_hat = model.predict(X_train, batch_size=ae_batch, verbose=0)
    X_test_hat = model.predict(X_test, batch_size=ae_batch, verbose=0)

    train_err = reconstruction_mae(X_train, X_train_hat)
    test_err = reconstruction_mae(X_test, X_test_hat)

    thr = float(np.quantile(train_err, thr_q))
    pred_win = (test_err >= thr).astype(int)

    # Window metrics (true window labels)
    metrics_win = compute_binary_metrics(y_test_win, pred_win)

    # Point scores/preds
    score_point = windows_scores_to_point_scores_max(test_starts, win_size, test_err, len(test_raw))
    pred_point = (np.nan_to_num(score_point, nan=-np.inf) >= thr).astype(int)
    metrics_point = compute_binary_metrics(y_test_point, pred_point)

    # Save
    row = {
        "dataset": dataset_name,
        "method": "lstm_ae",
        "threshold": thr,
        "thr_quantile": thr_q,
        **metrics_point,
    }
    append_csv_row(csv_path, row)

    save_json(
        out_dir / "lstm_ae_metrics.json",
        {
            **row,
            "window_metrics": metrics_win,
            "n_train_windows": int(len(X_train)),
            "n_test_windows": int(len(X_test)),
            "ae_params": config.AE_PARAMS,
        },
    )

    np.save(out_dir / "lstm_ae_scores_win.npy", test_err)
    np.save(out_dir / "lstm_ae_pred_win.npy", pred_win)

    np.save(out_dir / "lstm_ae_scores.npy", score_point)
    np.save(out_dir / "lstm_ae_pred.npy", pred_point)

    # Plots
    plot_signal(
        test_raw[z0:z1], y_test_point[z0:z1], pred_point[z0:z1],
        title=f"{dataset_name} - LSTM-AE (zoom)",
        save_path=out_dir / "lstm_ae_signal_zoom.png",
        x_offset=z0,
    )
    plot_scores(
        score_point[z0:z1],
        threshold=thr,
        true_labels=y_test_point[z0:z1],
        title=f"{dataset_name} - LSTM-AE scores (zoom)",
        save_path=out_dir / "lstm_ae_scores_zoom.png",
        x_offset=z0,
    )

Epoch 1/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 16s 39ms/step - loss: 0.5037 - val_loss: 0.3010
Epoch 2/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - loss: 0.2588 - val_loss: 0.1665
Epoch 3/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - loss: 0.1995 - val_loss: 0.1634
Epoch 4/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - loss: 0.1806 - val_loss: 0.1309
Epoch 5/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - loss: 0.1651 - val_loss: 0.1184
Epoch 6/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.1475 - val_loss: 0.1089
Epoch 7/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.1428 - val_loss: 0.1290
Epoch 8/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.1482 - val_loss: 0.1004
Epoch 9/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.1133 - val_loss: 0.0852
Epoch 10/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.1024 - val_loss: 0.0822
Epoch 11/15
352/352 ━━━━━━━━━━━━━━━━━━━━ 13s 37ms/step - loss: 0.0953 - val_loss: 0.0703
Epoch 12/15
352/352 ━━━━━━━━━━